In [2]:
import numpy as np
import numpy.ma as ma
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from habitablePlanets import habitable_planets

exoplanets = pd.read_csv("exoplanets3.csv", sep=",")
training_data = np.array(exoplanets[:])

'''print(training_data.shape)
print(training_data)'''

habitable_x = np.array([np.array(training_data[0, :])])
habitable_y = np.array([0])
uninhabitable_x = np.array([np.array(training_data[0, :])])
uninhabitable_y = np.array([0])

for i in training_data:
    if i[0] in habitable_planets:
        habitable_x = np.append(habitable_x, np.array([np.array(i)]), axis = 0)
        habitable_y = np.append(habitable_y, 1)
        

for i in training_data:
    if i[0] not in habitable_planets:
        uninhabitable_x = np.append(uninhabitable_x, np.array([np.array(i)]), axis = 0)
        uninhabitable_y = np.append(uninhabitable_y, 0)

        
training_x = np.append(habitable_x[:, 1:], uninhabitable_x[:, 1:], axis = 0)
training_y = np.append(habitable_y, uninhabitable_y, axis = 0)

habitable_x = habitable_x[:, 1:]
uninhabitable_x = uninhabitable_x[:, 1:]

from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter

training_x = np.asarray(training_x).astype(np.float32)
training_x = np.where(np.isnan(training_x), ma.array(training_x, mask=np.isnan(training_x)).mean(axis=0), training_x)

oversample = SMOTE()
x_res, y_res = oversample.fit_resample(training_x, training_y)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size = 0.2, random_state = 0)

#feature scaling
'''
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)'''

'\nfrom sklearn.preprocessing import StandardScaler\nsc = StandardScaler()\nx_train = sc.fit_transform(x_train)\nx_test = sc.fit_transform(x_test)'

In [10]:
%%time
from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(x_train, y_train)
print(LogReg.score(x_train, y_train))

0.8694718763417776
Wall time: 326 ms


C:\Users\Aland Liu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
%%time
y_pred = LogReg.predict(x_test)

Wall time: 3.99 ms


In [5]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[713 150]
 [121 763]]
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       863
           1       0.84      0.86      0.85       884

    accuracy                           0.84      1747
   macro avg       0.85      0.84      0.84      1747
weighted avg       0.85      0.84      0.84      1747

0.8448769318832284


In [7]:

#nonsynthetic data

x_test = np.array([np.array(training_data[0, 1:])])
y_test = np.array([0])

hab_lim = 20
cur_hab = 1

for i in training_data:
    if cur_hab >= hab_lim:
        break
    if i[0] in habitable_planets:
        y_test = np.append(y_test, 1)
        cur_hab += 1
    else:
        y_test = np.append(y_test, 0)
    x_test = np.append(x_test, np.array([np.array(i[1:])]), axis = 0)

x_test = np.asarray(x_test).astype(np.float32)
x_test = np.where(np.isnan(x_test), ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)
y_test = np.asarray(y_test).astype(np.float32)
y_test = np.where(np.isnan(y_test), ma.array(y_test, mask=np.isnan(y_test)).mean(axis=0), y_test)
    

In [8]:
%%time
y_pred = LogReg.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[1233  183]
 [   2   17]]
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93      1416
         1.0       0.09      0.89      0.16        19

    accuracy                           0.87      1435
   macro avg       0.54      0.88      0.54      1435
weighted avg       0.99      0.87      0.92      1435

0.8710801393728222
Wall time: 30.9 ms
